In [ ]:
                # This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
data = '/kaggle/input/leafcsvv/leaf.csv'
df = pd.read_csv(data)
df.head()

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
df.columns


In [ ]:
df['Class (Species)'].value_counts()

In [ ]:
#Xem số liệu sau khi mã hóa data
df.describe()

In [ ]:
plt.figure(figsize=(12, 6))
sns.heatmap(df.corr(), annot=True,cmap=plt.cm.cool)
plt.tight_layout()
plt.xlabel('Corr')
plt.show()

In [ ]:
X_dt = df.drop("Class (Species)" , axis = 1).values
Y = df['Class (Species)'].values
X_train_dt, X_test_dt, Y_train_dt, Y_test_dt = train_test_split(X_dt, Y, random_state=42, test_size=0.3) #train 70% - test 30%
print(X_train_dt.shape)
print(X_test_dt.shape)

In [ ]:
#có tác dụng chuyển đổi dữ liệu đào tạo và chỉ chuyển đổi dữ liệu kiểm tra
scaler_train= StandardScaler()
scaler_train.fit(X_train_dt)
scaler_test= StandardScaler()
scaler_test.fit(X_test_dt)
X_train_scaled= scaler_train.transform(X_train_dt)
X_test_scaled= scaler_test.transform(X_test_dt)

In [ ]:
from sklearn.tree import DecisionTreeClassifier
tree_model = DecisionTreeClassifier()
tree_model = tree_model.fit(X_train_dt,Y_train_dt)
y_pred = tree_model.predict(X_test_dt)

In [ ]:
from sklearn import metrics
y_pred = tree_model.predict(X_test_scaled)
y_pred = pd.DataFrame(y_pred)
MSE_tree = metrics.mean_squared_error(Y_test_dt, y_pred)
RMSE_tree =np.sqrt(MSE_tree)
print(RMSE_tree)

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, plot_confusion_matrix
from sklearn.tree import plot_tree
from sklearn.model_selection import cross_val_predict, cross_val_score

In [ ]:
dt = DecisionTreeClassifier(criterion= 'entropy',max_depth=11 , random_state=42)
dt.fit(X_train_scaled,Y_train_dt)
dt_pred = dt.predict(X_test_scaled)

print('Accuracy_Score of Decision Tree: ', accuracy_score(Y_test_dt, dt_pred))
matrix=confusion_matrix(Y_test_dt, dt_pred)

In [ ]:
#biểu đồ thống kê tỉ lệ cho từng loại lá
ax = sns.countplot(x='Class (Species)', data=df)

ax.set_xticklabels(ax.get_xticklabels(), fontsize=7)
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(10,10), dpi=150)
plot_tree(tree_model, filled=True, feature_names=df.columns,fontsize=10);
plt.savefig('imag.jpg')

#thuật toán KNN

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, plot_confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_curve, plot_precision_recall_curve, plot_roc_curve
from sklearn import metrics
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay


In [ ]:
#Đánh giá độ chính xác của mô hình
test_error_rate= []


for k in range (1, 10):
    knn_model = KNeighborsClassifier(n_neighbors=k)
    knn_model.fit(X_train_scaled, Y_train_dt)
    
    y_pred_test = knn_model.predict(X_test_scaled)
    
    test_error=1- accuracy_score(Y_test_dt, y_pred_test)
    test_error_rate.append(test_error)

In [ ]:
plt.figure(figsize=(8, 7))
plt.plot(range(1, 10), test_error_rate, label='Test Error')
plt.legend()
plt.ylabel('Error Rate')
plt.xlabel('K Value')

In [ ]:
# hệ số k láng giềng và hàm khoảng cách 
knn_model = KNeighborsClassifier(n_neighbors=4)
knn_model.fit(X_train_scaled, Y_train_dt) 
#Thực hiện dự đoán trên bộ dữ liệu kiểm tra.
y_pred = knn_model.predict(X_test_scaled) 
y_pred = pd.DataFrame(dt_pred)
MSE = metrics.mean_squared_error(Y_test_dt, dt_pred)
RMSE =np.sqrt(MSE)
print(RMSE)

In [ ]:
accuracy=metrics.accuracy_score(Y_test_dt,dt_pred)
print("\nĐộ chính xác của mô hình:", accuracy)

In [ ]:
#Độ chính xác cân bằng
model_balanced_acc = metrics.balanced_accuracy_score(Y_test_dt, dt_pred)
print("Độ chính xác cân bằng của mô hình:", model_balanced_acc)

In [ ]:
#ma trận hỗn hợp
confusion_matrix(Y_test_dt,dt_pred)

In [ ]:
# heatmap
plt.figure(figsize=(12, 6))
sns = sns.heatmap(confusion_matrix, annot=True, cmap='nipy_spectral_r', fmt='g')
sns.set_title("Ma trận lỗi")
plt.show()

In [ ]:
print(classification_report(Y_test_dt, y_pred))